<a href="https://www.kaggle.com/code/tiheli/audi-price-prediction-without-train-test-split?scriptVersionId=206121103" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/audi-a1-listings/Audi_A1_listings.csv


In [2]:
# Loading the dataset from a CSV file containing Audi A1 car listings
df = pd.read_csv('/kaggle/input/audi-a1-listings/Audi_A1_listings.csv')

In [3]:
# Displaying the first few rows of the dataset to inspect its structure and contents
df.head()

,index,Year,Type,Mileage(miles),Engine,PS,Transmission,Fuel,Number_of_Owners,Price(£),href,PPY,MileageRank,PriceRank,PPYRank,Score
0,0,2018.0,Hatchback,44000.0,1.6L,114.398422,Manual,Diesel,1,14995.0,https://www.autotrader.co.uk/car-details/20221...,2499.166667,215,163,340,718
1,4,2016.0,Hatchback,42596.0,1.0L,93.688363,Manual,Petrol,3,10755.0,https://www.autotrader.co.uk/car-details/20221...,2688.750000,222,330,276,828
2,7,2015.0,Hatchback,42700.0,1.4L,123.274162,Manual,Petrol,2,10799.0,https://www.autotrader.co.uk/car-details/20221...,3599.666667,221,327,94,642
3,11,2014.0,Hatchback,86000.0,1.6L,103.550296,Manual,Diesel,3,7490.0,https://www.autotrader.co.uk/car-details/20221...,3745.000000,41,449,83,573
4,12,2014.0,Hatchback,104310.0,1.6L,103.550296,Manual,Diesel,3,7400.0,https://www.autotrader.co.uk/car-details/20220...,3700.000000,12,452,85,549


In [4]:
# Dropping unnecessary columns that do not contribute to the analysis or model training
df = df.drop(columns=['index', 'href', 'MileageRank', 'PriceRank', 'PPYRank', 'Score'])

In [5]:
df.head(3)

,Year,Type,Mileage(miles),Engine,PS,Transmission,Fuel,Number_of_Owners,Price(£),PPY
0,2018.0,Hatchback,44000.0,1.6L,114.398422,Manual,Diesel,1,14995.0,2499.166667
1,2016.0,Hatchback,42596.0,1.0L,93.688363,Manual,Petrol,3,10755.0,2688.750000
2,2015.0,Hatchback,42700.0,1.4L,123.274162,Manual,Petrol,2,10799.0,3599.666667


In [6]:
# Displaying a summary of the dataset to check data types, non-null counts, and overall structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              471 non-null    float64
 1   Type              471 non-null    object 
 2   Mileage(miles)    471 non-null    float64
 3   Engine            471 non-null    object 
 4   PS                471 non-null    float64
 5   Transmission      471 non-null    object 
 6   Fuel              471 non-null    object 
 7   Number_of_Owners  471 non-null    int64  
 8   Price(£)          471 non-null    float64
 9   PPY               471 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 36.9+ KB


In [7]:
# Removing the "L" character from the 'Engine' column values to keep only numeric data for engine size
df['Engine'] = df['Engine'].str.replace("L", "")

In [8]:
# Converting the 'Engine' column to a numeric data type for easier analysis and computation
df['Engine'] = pd.to_numeric(df['Engine'])

In [9]:
df.head(3)

,Year,Type,Mileage(miles),Engine,PS,Transmission,Fuel,Number_of_Owners,Price(£),PPY
0,2018.0,Hatchback,44000.0,1.6,114.398422,Manual,Diesel,1,14995.0,2499.166667
1,2016.0,Hatchback,42596.0,1.0,93.688363,Manual,Petrol,3,10755.0,2688.750000
2,2015.0,Hatchback,42700.0,1.4,123.274162,Manual,Petrol,2,10799.0,3599.666667


In [10]:
# Encoding categorical columns ('Type', 'Transmission', 'Fuel') using one-hot encoding,
# dropping the first category in each to avoid multicollinearity in regression models
df = pd.get_dummies(df, columns=['Type', 'Transmission', 'Fuel'], drop_first=True)

In [11]:
# Importing the LinearRegression model to build and train a regression model on the dataset
from sklearn.linear_model import LinearRegression

In [12]:
# Setting 'Price(£)' as the target variable (y) for the model, representing the price of the car
# Dropping 'Price(£)' from the features (x) to use the remaining columns for prediction
y = df[['Price(£)']]
x = df.drop("Price(£)", axis=1)

In [13]:
# Initializing a LinearRegression model and fitting it to the data
# The model is trained to predict car prices (y) based on the features (x)
lm = LinearRegression()
model = lm.fit(x, y)

In [14]:
# Using the trained model to predict the price of a car with specified feature values
# The input features correspond to the car's year, mileage, engine size, horsepower, etc.
model.predict([[2017, 30000, 1.6, 110, 1, 2600, 0, 1]])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[13145.9080419]])

In [15]:
model.predict([[2015, 45000, 1.2, 95, 0, 2200, 1, 0]])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[7667.26890422]])

In [16]:
# Evaluating the model's performance by calculating the R-squared score on the training data
# The score represents how well the model explains the variance in the target variable (price)
model.score(x, y)

0.9595295772839731